<a href="https://colab.research.google.com/github/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_09_3_transfer_cv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
!pip install pyyaml h5py

In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import tensorflow.keras
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
#from tensorflow.keras.applicNoations import MobileNet, InceptionV3
from keras.applications.vgg16 import VGG16
#from tensorflow.keras.applications.inception_v3 import InceptionV3 as MN
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
#from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
SIZE = 224 #Resize images

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
base_model=VGG16(weights='imagenet',include_top=False) 

In [ ]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [ ]:
for layer in base_model.layers:
	layer.trainable = False

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) 
x=Dense(1024,activation='relu')(x) 
preds=Dense(3,activation='softmax')(x) 

In [ ]:
model=Model(inputs=base_model.input,outputs=preds)
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_4 ( (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dense_9 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_10 (Dense)             (None, 3)                 3075      
Total params: 16,292,675
Trainable params: 1,577,987
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
PATH = "/content/drive/MyDrive/BCFILES/images/train"

data_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,
#                                               rotation_range=180,width_shift_range=0.2,height_shift_range=0.2,
#                                                shear_range=0.2,horizontal_flip=True,,fill_mode='nearest' 
                                              #   rotation_range=180,                                               
                                                 zoom_range=0.2,
                                                 validation_split=0.25                                                                                            
                                )
                                  
train_generator=data_datagen.flow_from_directory(PATH, 
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=52,
                                                 class_mode='categorical',
                                                 shuffle=True,
                                                 subset='training'
                                                  )
val_generator = data_datagen.flow_from_directory(
                                                  PATH,
                                                  subset='validation',
                                                  target_size=(224,224),
                                                  color_mode='rgb',
                                                  batch_size=52,
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  )


Found 311 images belonging to 3 classes.
Found 102 images belonging to 3 classes.


In [ ]:
ACCURACY_THRESHOLD = 0.88

class myCallback(tf.keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('val_accuracy') >= ACCURACY_THRESHOLD):   
          print("\nReached %2.2f%% accuracy, so stopping training!!" %(ACCURACY_THRESHOLD*100))   
          self.model.stop_training = True
callbacks = myCallback()

In [ ]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',
              metrics=['accuracy'])
 
step_size_train=train_generator.n//train_generator.batch_size
'''
history = model.fit(train_generator, epochs=50, validation_data = validation_generator,
                     steps_per_epoch=step_size_train,
                     workers=10,
                     callbacks=[callbacks]
                    )'''
 
 
history = model.fit(train_generator, epochs=50, validation_data = val_generator,
      #              steps_per_epoch=8,
      #               workers=2,
      #              callbacks=[callbacks]
                    )

Epoch 1/50


FailedPreconditionError: ignored

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()
 
 
plt.show()

In [ ]:

#model.save('/content/drive/MyDrive/BCFILES/vgg16102.h5')

In [ ]:
PATHv = "/content/drive/MyDrive/BCFILES/images/evaluation"

eva_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,
                                    horizontal_flip=True,
                                    
                                        #,rotation_range=180
                                    #         ,width_shift_range=0.2,height_shift_range=0.2,
                                   #          shear_range=0.2,,fill_mode='nearest'
                                       )                                                           
eva_generator=eva_datagen.flow_from_directory(PATHv, 
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=52,
                                                 class_mode='categorical',
                                                 shuffle=True #######
                                      ) 

Found 22 images belonging to 3 classes.


In [ ]:
print(eva_generator.class_indices)

model.evaluate(eva_generator)

In [ ]:
from google.colab import files


uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(224, 224))
#  x = image.img_to_array(img)
#  x = np.expand_dims(x, axis=0)
#  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(fn)
  print(classes)
  pred2 = int(np.argmax(classes,axis=1))
  print('Grade:',pred2+1) 

In [ ]:
model.save('/content/drive/MyDrive/BCFILES/')

In [ ]:
from tensorflow import keras

reconstructed_model = keras.models.load_model("/content/drive/MyDrive/BCFILES/vgg16102.h5")

In [ ]:
print(eva_generator.class_indices, '\n')

reconstructed_model.evaluate(eva_generator, workers=2)

{'G1': 0, 'G2': 1, 'G3': 2} 

1/1 [==============================] - 0s 424ms/step - loss: 2.9502 - accuracy: 0.7273


[2.950169563293457, 0.7272727489471436]